In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
def percent(x, y):
    return (x/y)*100

## This notebook outlines the high level industry data formatting for the Comprehensive Plans with place level primary geographies. There is another document for the information that is pulled for all relevant geographies.

The Woods & Poole data that is used more at the county level is only used at the place level for total employment projections. This is for consistency. This is a land use model ultimately, so JobsEQ employment is selected to model place level employment for place of work. This data is derived from the QCEW and other inputs so is selected as a better option - both are modeled so we use the one modeled directly from UI claims rather than land use at this granular of a geography.

To retrieve data:
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ go to "Draft Mode"
+ Select the "Occupation" dataset
+ Select the following dimensions: Region, Industry  
+ Select the following values: "Employment (Place of Work)", "Employment (Place of Residence)", "Annual Wage (Mean)", "Annual Wage (Median)", "Location Quotient", Demand - Total(Forecast) (3 Years), Demand - Growth Rate (Forecast)(3 Years)"
+ Under "Region" select "USA", "Tennessee", then "GNRC Region" and sub all "Counties" and "Census Places"   
+ Under "Occupation" expand "Total-All Industries" and select 2 digit SOCS codes, leaving Total as well

Once it is completed, open and delete the top row that says "Industry Data", and delete the FIPS column. Rename the Region column to NAME, delete the data notes at the bottom.  "JobsEQOccupationalEmployment_Place.csv"

In [92]:
data = pd.read_csv('../../Data Downloads/JobsEQOccupationalEmployment_allgeos.csv')

In [93]:
data.head()

,NAME,Total - All Occupations,Total - All Occupations.1,Total - All Occupations.2,Total - All Occupations.3,Total - All Occupations.4,Total - All Occupations.5,Total - All Occupations.6,Total - All Occupations.7,Healthcare Practitioners and Technical Occupations (29-0000),Healthcare Practitioners and Technical Occupations (29-0000).1,Healthcare Practitioners and Technical Occupations (29-0000).2,Healthcare Practitioners and Technical Occupations (29-0000).3,Healthcare Practitioners and Technical Occupations (29-0000).4,Healthcare Practitioners and Technical Occupations (29-0000).5,Healthcare Practitioners and Technical Occupations (29-0000).6,Healthcare Practitioners and Technical Occupations (29-0000).7,Healthcare Support Occupations (31-0000),Healthcare Support Occupations (31-0000).1,Healthcare Support Occupations (31-0000).2,Healthcare Support Occupations (31-0000).3,Healthcare Support Occupations (31-0000).4,Healthcare Support Occupations (31-0000).5,Healthcare Support Occupations (31-0000).6,Healthcare Support Occupations (31-0000).7,Protective Service Occupations (33-0000),Protective Service Occupations (33-0000).1,Protective Service Occupations (33-0000).2,Protective Service Occupations (33-0000).3,Protective Service Occupations (33-0000).4,Protective Service Occupations (33-0000).5,Protective Service Occupations (33-0000).6,Protective Service Occupations (33-0000).7,Production Occupations (51-0000),Production Occupations (51-0000).1,Production Occupations (51-0000).2,Production Occupations (51-0000).3,Production Occupations (51-0000).4,Production Occupations (51-0000).5,Production Occupations (51-0000).6,Production Occupations (51-0000).7,Transportation and Material Moving Occupations (53-0000),Transportation and Material Moving Occupations (53-0000).1,Transportation and Material Moving Occupations (53-0000).2,Transportation and Material Moving Occupations (53-0000).3,Transportation and Material Moving Occupations (53-0000).4,Transportation and Material Moving Occupations (53-0000).5,Transportation and Material Moving Occupations (53-0000).6,Transportation and Material Moving Occupations (53-0000).7,Business & Financial,Business & Financial.1,Business & Financial.2,Business & Financial.3,Business & Financial.4,Business & Financial.5,Business & Financial.6,Business & Financial.7,"Computer, Mathematical, & Engineering","Computer, Mathematical, & Engineering.1","Computer, Mathematical, & Engineering.2","Computer, Mathematical, & Engineering.3","Computer, Mathematical, & Engineering.4","Computer, Mathematical, & Engineering.5","Computer, Mathematical, & Engineering.6","Computer, Mathematical, & Engineering.7","Education, Legal, Social, Arts, & Media","Education, Legal, Social, Arts, & Media.1","Education, Legal, Social, Arts, & Media.2","Education, Legal, Social, Arts, & Media.3","Education, Legal, Social, Arts, & Media.4","Education, Legal, Social, Arts, & Media.5","Education, Legal, Social, Arts, & Media.6","Education, Legal, Social, Arts, & Media.7","Food, Property, & Personal Services","Food, Property, & Personal Services.1","Food, Property, & Personal Services.2","Food, Property, & Personal Services.3","Food, Property, & Personal Services.4","Food, Property, & Personal Services.5","Food, Property, & Personal Services.6","Food, Property, & Personal Services.7",Sales & Office Support,Sales & Office Support.1,Sales & Office Support.2,Sales & Office Support.3,Sales & Office Support.4,Sales & Office Support.5,Sales & Office Support.6,Sales & Office Support.7,"Natural Resources, Construction, & Maintenance","Natural Resources, Construction, & Maintenance.1","Natural Resources, Construction, & Maintenance.2","Natural Resources, Construction, & Maintenance.3","Natural Resources, Construction, & Maintenance.4","Natural Resources, Construction, & Maintenance.5","Natural Resources, Construction, & Maintenance.6","Natural Resources, Construction, & Maintenance.7"
0,NAME,Empl (Place of Work),Empl (Place of Residence),Annual Mean 

In [94]:
data = data.set_index('NAME').transpose()
data.head()

NAME,NAME,USA,Tennessee,GNRC Region,"Adams city, TN","Ashland City town, TN","Belle Meade city, TN","Berry Hill city, TN","Bethpage CDP, TN","Bransford CDP, TN","Brentwood city, TN","Burns town, TN","Castalian Springs CDP, TN","Cedar Hill city, TN","Charlotte town, TN","Christiana CDP, TN","Clarksville city, TN","Columbia city, TN","Coopertown town, TN","Cottontown CDP, TN","Cross Plains city, TN","Cumberland City town, TN","Dickson city, TN","Dover city, TN","Eagleville city, TN","Erin city, TN","Fairfield CDP, TN","Fairview city, TN","Forest Hills city, TN","Franklin city, TN","Gallatin city, TN","Goodlettsville city, TN","Graball CDP, TN","Greenbrier town, TN","Green Hill CDP, TN","Hartsville/Trousdale County, TN","Hendersonville city, TN","Kingston Springs town, TN","La Vergne city, TN","Lebanon city, TN","McEwen city, TN","Millersville city, TN","Mitchellville city, TN","Mount Juliet city, TN","Mount Pleasant city, TN","Murfreesboro city, TN","Nashville-Davidson metropolitan government (balance), TN","New Deal CDP, TN","New Johnsonville city, TN","Nolensville town, TN","Oak Grove CDP, TN","Oak Hill city, TN","Orlinda city, TN","Pegram town, TN","Pleasant View city, TN","Portland city, TN","Ridgetop city, TN","Rockvale CDP, TN","Rural Hill CDP, TN","Shackle Island CDP, TN","Slayden town, TN","Smyrna town, TN","Springfield city, TN","Spring Hill city, TN","Tennessee Ridge town, TN","Thompson's Station town, TN","Vanleer town, TN","Walnut Grove CDP, TN","Walterhill CDP, TN","Watertown city, TN","Waverly city, TN","Westmoreland town, TN","White Bluff town, TN","White House city, TN","Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee",MPO,GNRC
Total - All Occupations,Empl (Place of Work),160228834,3393466,1179322,67,5004,1319,18306,66,7,57368,194,45,64,1029,378,54378,22584,1700,20,295,820,12897,995,481,902,1,1886,897,82917,23352,14016,2,1249,680,2302,25332,903,18354,34322,509,1397,28,18180,2159,76307,524693,58,649,2768,61,1223,444,298,1602,7850,248,126,172,456,17,41763,11300,10403,231,1751,53,49,204,560,2151,899,935,5011,10901,558576,20319,1784,6502,41515,64976,25726,149974,3331,66405,2302,160642,66367,1069207,1137806
Total - All Occupations.1,Empl (Place of Residence),155375879,3243461,1170962,383,2336,1418,737,169,93,27147,832,366,205,551,2676,68456,21540,1952,263,1270,141,7220,708,502,413,64,6274,2433,49972,19255,9121,117,3799,3711,6134,32881,1461,18063,18492,641,3497,129,19610,2538,77785,397873,192,718,7347,131,2715,471,1025,2420,6120,1045,616,1623,1868,52,30897,8424,24574,573,2053,207,493,310,860,1544,1198,1926,6639,21098,411141,24987,3046,7093,50998,89593,36375,183759,5327,100015,6134,147219,84177,1013684,1119964
Total - All Occupations.2,Annual Mean Wage1,61600,53200,56400,48500,50300,60400,60400,50100,50100,62900,46300,50100,48500,46300,50300,48200,48900,48500,50100,48500,51800,46300,51800,50300,42100,50100,62900,60400,62900,50100,59000,50100,48500,48300,45900,50100,50300,50300,48300,47900,49500,50100,48300,48900,50300,60400,50100,47900,62900,50100,60400,48500,50300,50300,50000,48500,50300,48300,50100,46300,50300,48500,54400,42100,62900,46300,50100,50300,48300,47900,50100,46300,48800,50300,60400,46300,42100,47900,48900,48200,48500,50300,51800,50100,45900,62900,48300,57200,56600
Total - All Occupations.3,Ann 50th %ile Wage1,45700,40900,43000,38700,40500,45300,45300,39000,39000,46800,36500,39000,38700,36500,39600,38000,38500,38700,39000,38700,42100,36500,42100,39600,35500,39000,46800,45300,46800,39000,44500,39000,38700,38300,37000,39000,40500,39600,38300,39500,38900,39000,38300,38500,39600,45300,39000,39500,46800,39000,45300,38700,40500,40500,39000,38700,39600,38300,39000,36

In [95]:
data = data.rename(columns = {'NAME': 'Value'})

In [96]:
data = data.reset_index(drop = False)

In [97]:
data.head()

NAME,index,Value,USA,Tennessee,GNRC Region,"Adams city, TN","Ashland City town, TN","Belle Meade city, TN","Berry Hill city, TN","Bethpage CDP, TN","Bransford CDP, TN","Brentwood city, TN","Burns town, TN","Castalian Springs CDP, TN","Cedar Hill city, TN","Charlotte town, TN","Christiana CDP, TN","Clarksville city, TN","Columbia city, TN","Coopertown town, TN","Cottontown CDP, TN","Cross Plains city, TN","Cumberland City town, TN","Dickson city, TN","Dover city, TN","Eagleville city, TN","Erin city, TN","Fairfield CDP, TN","Fairview city, TN","Forest Hills city, TN","Franklin city, TN","Gallatin city, TN","Goodlettsville city, TN","Graball CDP, TN","Greenbrier town, TN","Green Hill CDP, TN","Hartsville/Trousdale County, TN","Hendersonville city, TN","Kingston Springs town, TN","La Vergne city, TN","Lebanon city, TN","McEwen city, TN","Millersville city, TN","Mitchellville city, TN","Mount Juliet city, TN","Mount Pleasant city, TN","Murfreesboro city, TN","Nashville-Davidson metropolitan government (balance), TN","New Deal CDP, TN","New Johnsonville city, TN","Nolensville town, TN","Oak Grove CDP, TN","Oak Hill city, TN","Orlinda city, TN","Pegram town, TN","Pleasant View city, TN","Portland city, TN","Ridgetop city, TN","Rockvale CDP, TN","Rural Hill CDP, TN","Shackle Island CDP, TN","Slayden town, TN","Smyrna town, TN","Springfield city, TN","Spring Hill city, TN","Tennessee Ridge town, TN","Thompson's Station town, TN","Vanleer town, TN","Walnut Grove CDP, TN","Walterhill CDP, TN","Watertown city, TN","Waverly city, TN","Westmoreland town, TN","White Bluff town, TN","White House city, TN","Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee",MPO,GNRC
0,Total - All Occupations,Empl (Place of Work),160228834,3393466,1179322,67,5004,1319,18306,66,7,57368,194,45,64,1029,378,54378,22584,1700,20,295,820,12897,995,481,902,1,1886,897,82917,23352,14016,2,1249,680,2302,25332,903,18354,34322,509,1397,28,18180,2159,76307,524693,58,649,2768,61,1223,444,298,1602,7850,248,126,172,456,17,41763,11300,10403,231,1751,53,49,204,560,2151,899,935,5011,10901,558576,20319,1784,6502,41515,64976,25726,149974,3331,66405,2302,160642,66367,1069207,1137806
1,Total - All Occupations.1,Empl (Place of Residence),155375879,3243461,1170962,383,2336,1418,737,169,93,27147,832,366,205,551,2676,68456,21540,1952,263,1270,141,7220,708,502,413,64,6274,2433,49972,19255,9121,117,3799,3711,6134,32881,1461,18063,18492,641,3497,129,19610,2538,77785,397873,192,718,7347,131,2715,471,1025,2420,6120,1045,616,1623,1868,52,30897,8424,24574,573,2053,207,493,310,860,1544,1198,1926,6639,21098,411141,24987,3046,7093,50998,89593,36375,183759,5327,100015,6134,147219,84177,1013684,1119964
2,Total - All Occupations.2,Annual Mean Wage1,61600,53200,56400,48500,50300,60400,60400,50100,50100,62900,46300,50100,48500,46300,50300,48200,48900,48500,50100,48500,51800,46300,51800,50300,42100,50100,62900,60400,62900,50100,59000,50100,48500,48300,45900,50100,50300,50300,48300,47900,49500,50100,48300,48900,50300,60400,50100,47900,62900,50100,60400,48500,50300,50300,50000,48500,50300,48300,50100,46300,50300,48500,54400,42100,62900,46300,50100,50300,48300,47900,50100,46300,48800,50300,60400,46300,42100,47900,48900,48200,48500,50300,51800,50100,45900,62900,48300,57200,56600
3,Total - All Occupations.3,Ann 50th %ile Wage1,45700,40900,43000,38700,40500,45300,45300,39000,39000,46800,36500,39000,38700,36500,39600,38000,38500,38700,39000,38700,42100,36500,42100,39600,35500,39000,46800,45300,46800,39000,44500,39000,38700,38300,37000,39000,40500,39600,38300,39500,38900,39000,38300,38500,39600,45300,39000,39500,46800,39000,45300,38700,40500,40500,39000,38700,39600

In [98]:
data['index'] = data['index'].str.strip()
data['Value'] = data['Value'].str.strip()

In [99]:
occs = ['Healthcare Practitioners and Technical Occupations (29-0000)', 'Healthcare Support Occupations (31-0000)', 'Protective Service Occupations (33-0000)',
        'Production Occupations (51-0000)', 'Transportation and Material Moving Occupations (53-0000)', 'Business & Financial', 
        'Computer, Mathematical, & Engineering', 'Education, Legal, Social, Arts, & Media', 'Food, Property, & Personal Services', 
        'Sales & Office Support', 'Natural Resources, Construction, & Maintenance']

In [100]:
test = data.set_index(['index', 'Value']).transpose()
test.head()

index,Total - All Occupations,Total - All Occupations.1,Total - All Occupations.2,Total - All Occupations.3,Total - All Occupations.4,Total - All Occupations.5,Total - All Occupations.6,Total - All Occupations.7,Healthcare Practitioners and Technical Occupations (29-0000),Healthcare Practitioners and Technical Occupations (29-0000).1,Healthcare Practitioners and Technical Occupations (29-0000).2,Healthcare Practitioners and Technical Occupations (29-0000).3,Healthcare Practitioners and Technical Occupations (29-0000).4,Healthcare Practitioners and Technical Occupations (29-0000).5,Healthcare Practitioners and Technical Occupations (29-0000).6,Healthcare Practitioners and Technical Occupations (29-0000).7,Healthcare Support Occupations (31-0000),Healthcare Support Occupations (31-0000).1,Healthcare Support Occupations (31-0000).2,Healthcare Support Occupations (31-0000).3,Healthcare Support Occupations (31-0000).4,Healthcare Support Occupations (31-0000).5,Healthcare Support Occupations (31-0000).6,Healthcare Support Occupations (31-0000).7,Protective Service Occupations (33-0000),Protective Service Occupations (33-0000).1,Protective Service Occupations (33-0000).2,Protective Service Occupations (33-0000).3,Protective Service Occupations (33-0000).4,Protective Service Occupations (33-0000).5,Protective Service Occupations (33-0000).6,Protective Service Occupations (33-0000).7,Production Occupations (51-0000),Production Occupations (51-0000).1,Production Occupations (51-0000).2,Production Occupations (51-0000).3,Production Occupations (51-0000).4,Production Occupations (51-0000).5,Production Occupations (51-0000).6,Production Occupations (51-0000).7,Transportation and Material Moving Occupations (53-0000),Transportation and Material Moving Occupations (53-0000).1,Transportation and Material Moving Occupations (53-0000).2,Transportation and Material Moving Occupations (53-0000).3,Transportation and Material Moving Occupations (53-0000).4,Transportation and Material Moving Occupations (53-0000).5,Transportation and Material Moving Occupations (53-0000).6,Transportation and Material Moving Occupations (53-0000).7,Business & Financial,Business & Financial.1,Business & Financial.2,Business & Financial.3,Business & Financial.4,Business & Financial.5,Business & Financial.6,Business & Financial.7,"Computer, Mathematical, & Engineering","Computer, Mathematical, & Engineering.1","Computer, Mathematical, & Engineering.2","Computer, Mathematical, & Engineering.3","Computer, Mathematical, & Engineering.4","Computer, Mathematical, & Engineering.5","Computer, Mathematical, & Engineering.6","Computer, Mathematical, & Engineering.7","Education, Legal, Social, Arts, & Media","Education, Legal, Social, Arts, & Media.1","Education, Legal, Social, Arts, & Media.2","Education, Legal, Social, Arts, & Media.3","Education, Legal, Social, Arts, & Media.4","Education, Legal, Social, Arts, & Media.5","Education, Legal, Social, Arts, & Media.6","Education, Legal, Social, Arts, & Media.7","Food, Property, & Personal Services","Food, Property, & Personal Services.1","Food, Property, & Personal Services.2","Food, Property, & Personal Services.3","Food, Property, & Personal Services.4","Food, Property, & Personal Services.5","Food, Property, & Personal Services.6","Food, Property, & Personal Services.7",Sales & Office Support,Sales & Office Support.1,Sales & Office Support.2,Sales & Office Support.3,Sales & Office Support.4,Sales & Office Support.5,Sales & Office Support.6,Sales & Office Support.7,"Natural Resources, Construction, & Maintenance","Natural Resources, Construction, & Maintenance.1","Natural Resources, Construction, & Maintenance.2","Natural Resources, Construction, & Maintenance.3","Natural Resources, Construction, & Maintenance.4","Natural Resources, Construction, & Maintenance.5","Natural Resources, Construction, & Maintenance.6","Natural Resources, Construction, & Maintenance.7"
Value,Empl (Place of Work),Empl (Place of Residence),Annual Mean W

In [101]:
test = test.fillna(0)

In [102]:
test.head(2)

index,Total - All Occupations,Total - All Occupations.1,Total - All Occupations.2,Total - All Occupations.3,Total - All Occupations.4,Total - All Occupations.5,Total - All Occupations.6,Total - All Occupations.7,Healthcare Practitioners and Technical Occupations (29-0000),Healthcare Practitioners and Technical Occupations (29-0000).1,Healthcare Practitioners and Technical Occupations (29-0000).2,Healthcare Practitioners and Technical Occupations (29-0000).3,Healthcare Practitioners and Technical Occupations (29-0000).4,Healthcare Practitioners and Technical Occupations (29-0000).5,Healthcare Practitioners and Technical Occupations (29-0000).6,Healthcare Practitioners and Technical Occupations (29-0000).7,Healthcare Support Occupations (31-0000),Healthcare Support Occupations (31-0000).1,Healthcare Support Occupations (31-0000).2,Healthcare Support Occupations (31-0000).3,Healthcare Support Occupations (31-0000).4,Healthcare Support Occupations (31-0000).5,Healthcare Support Occupations (31-0000).6,Healthcare Support Occupations (31-0000).7,Protective Service Occupations (33-0000),Protective Service Occupations (33-0000).1,Protective Service Occupations (33-0000).2,Protective Service Occupations (33-0000).3,Protective Service Occupations (33-0000).4,Protective Service Occupations (33-0000).5,Protective Service Occupations (33-0000).6,Protective Service Occupations (33-0000).7,Production Occupations (51-0000),Production Occupations (51-0000).1,Production Occupations (51-0000).2,Production Occupations (51-0000).3,Production Occupations (51-0000).4,Production Occupations (51-0000).5,Production Occupations (51-0000).6,Production Occupations (51-0000).7,Transportation and Material Moving Occupations (53-0000),Transportation and Material Moving Occupations (53-0000).1,Transportation and Material Moving Occupations (53-0000).2,Transportation and Material Moving Occupations (53-0000).3,Transportation and Material Moving Occupations (53-0000).4,Transportation and Material Moving Occupations (53-0000).5,Transportation and Material Moving Occupations (53-0000).6,Transportation and Material Moving Occupations (53-0000).7,Business & Financial,Business & Financial.1,Business & Financial.2,Business & Financial.3,Business & Financial.4,Business & Financial.5,Business & Financial.6,Business & Financial.7,"Computer, Mathematical, & Engineering","Computer, Mathematical, & Engineering.1","Computer, Mathematical, & Engineering.2","Computer, Mathematical, & Engineering.3","Computer, Mathematical, & Engineering.4","Computer, Mathematical, & Engineering.5","Computer, Mathematical, & Engineering.6","Computer, Mathematical, & Engineering.7","Education, Legal, Social, Arts, & Media","Education, Legal, Social, Arts, & Media.1","Education, Legal, Social, Arts, & Media.2","Education, Legal, Social, Arts, & Media.3","Education, Legal, Social, Arts, & Media.4","Education, Legal, Social, Arts, & Media.5","Education, Legal, Social, Arts, & Media.6","Education, Legal, Social, Arts, & Media.7","Food, Property, & Personal Services","Food, Property, & Personal Services.1","Food, Property, & Personal Services.2","Food, Property, & Personal Services.3","Food, Property, & Personal Services.4","Food, Property, & Personal Services.5","Food, Property, & Personal Services.6","Food, Property, & Personal Services.7",Sales & Office Support,Sales & Office Support.1,Sales & Office Support.2,Sales & Office Support.3,Sales & Office Support.4,Sales & Office Support.5,Sales & Office Support.6,Sales & Office Support.7,"Natural Resources, Construction, & Maintenance","Natural Resources, Construction, & Maintenance.1","Natural Resources, Construction, & Maintenance.2","Natural Resources, Construction, & Maintenance.3","Natural Resources, Construction, & Maintenance.4","Natural Resources, Construction, & Maintenance.5","Natural Resources, Construction, & Maintenance.6","Natural Resources, Construction, & Maintenance.7"
Value,Empl (Place of Work),Empl (Place of Residence),Annual Mean W

In [103]:
cols = test.columns
test[cols] = test[cols].astype(float)

In [104]:
for occ in occs:
    test[('{}'.format(occ), 'Percent of Total')] = percent(test[('{}'.format(occ), 'Empl (Place of Work)')], 
                                                           test[('Total - All Occupations', 'Empl (Place of Work)')])

C:\Users\jmccall\AppData\Local\Temp\ipykernel_14512\1688469197.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test[('{}'.format(occ), 'Percent of Total')] = percent(test[('{}'.format(occ), 'Empl (Place of Work)')],


In [105]:
test.head()

index,Total - All Occupations,Total - All Occupations.1,Total - All Occupations.2,Total - All Occupations.3,Total - All Occupations.4,Total - All Occupations.5,Total - All Occupations.6,Total - All Occupations.7,Healthcare Practitioners and Technical Occupations (29-0000),Healthcare Practitioners and Technical Occupations (29-0000).1,Healthcare Practitioners and Technical Occupations (29-0000).2,Healthcare Practitioners and Technical Occupations (29-0000).3,Healthcare Practitioners and Technical Occupations (29-0000).4,Healthcare Practitioners and Technical Occupations (29-0000).5,Healthcare Practitioners and Technical Occupations (29-0000).6,Healthcare Practitioners and Technical Occupations (29-0000).7,Healthcare Support Occupations (31-0000),Healthcare Support Occupations (31-0000).1,Healthcare Support Occupations (31-0000).2,Healthcare Support Occupations (31-0000).3,Healthcare Support Occupations (31-0000).4,Healthcare Support Occupations (31-0000).5,Healthcare Support Occupations (31-0000).6,Healthcare Support Occupations (31-0000).7,Protective Service Occupations (33-0000),Protective Service Occupations (33-0000).1,Protective Service Occupations (33-0000).2,Protective Service Occupations (33-0000).3,Protective Service Occupations (33-0000).4,Protective Service Occupations (33-0000).5,Protective Service Occupations (33-0000).6,Protective Service Occupations (33-0000).7,Production Occupations (51-0000),Production Occupations (51-0000).1,Production Occupations (51-0000).2,Production Occupations (51-0000).3,Production Occupations (51-0000).4,Production Occupations (51-0000).5,Production Occupations (51-0000).6,Production Occupations (51-0000).7,Transportation and Material Moving Occupations (53-0000),Transportation and Material Moving Occupations (53-0000).1,Transportation and Material Moving Occupations (53-0000).2,Transportation and Material Moving Occupations (53-0000).3,Transportation and Material Moving Occupations (53-0000).4,Transportation and Material Moving Occupations (53-0000).5,Transportation and Material Moving Occupations (53-0000).6,Transportation and Material Moving Occupations (53-0000).7,Business & Financial,Business & Financial.1,Business & Financial.2,Business & Financial.3,Business & Financial.4,Business & Financial.5,Business & Financial.6,Business & Financial.7,"Computer, Mathematical, & Engineering","Computer, Mathematical, & Engineering.1","Computer, Mathematical, & Engineering.2","Computer, Mathematical, & Engineering.3","Computer, Mathematical, & Engineering.4","Computer, Mathematical, & Engineering.5","Computer, Mathematical, & Engineering.6","Computer, Mathematical, & Engineering.7","Education, Legal, Social, Arts, & Media","Education, Legal, Social, Arts, & Media.1","Education, Legal, Social, Arts, & Media.2","Education, Legal, Social, Arts, & Media.3","Education, Legal, Social, Arts, & Media.4","Education, Legal, Social, Arts, & Media.5","Education, Legal, Social, Arts, & Media.6","Education, Legal, Social, Arts, & Media.7","Food, Property, & Personal Services","Food, Property, & Personal Services.1","Food, Property, & Personal Services.2","Food, Property, & Personal Services.3","Food, Property, & Personal Services.4","Food, Property, & Personal Services.5","Food, Property, & Personal Services.6","Food, Property, & Personal Services.7",Sales & Office Support,Sales & Office Support.1,Sales & Office Support.2,Sales & Office Support.3,Sales & Office Support.4,Sales & Office Support.5,Sales & Office Support.6,Sales & Office Support.7,"Natural Resources, Construction, & Maintenance","Natural Resources, Construction, & Maintenance.1","Natural Resources, Construction, & Maintenance.2","Natural Resources, Construction, & Maintenance.3","Natural Resources, Construction, & Maintenance.4","Natural Resources, Construction, & Maintenance.5","Natural Resources, Construction, & Maintenance.6","Natural Resources, Construction, & Maintenance.7",Healthcare Practitioners and Technical Occupations (29-0000),Healt

In [106]:
test = test.transpose()

In [107]:
test = test.reset_index(drop = False)

In [108]:
boop = test['index'].str.split(pat = ".", expand = True)
test['Occupation'] = boop[0]

In [109]:
data = test

In [110]:
data = data.drop(columns = 'index')

In [111]:
cols = data.columns
data = data.melt(var_name = 'NAME', id_vars = ['Value', 'Occupation'], value_vars = cols, value_name = 'IDK')
data.head()

,Value,Occupation,NAME,IDK
0,Empl (Place of Work),Total - All Occupations,USA,160228834.0
1,Empl (Place of Residence),Total - All Occupations,USA,155375879.0
2,Annual Mean Wage1,Total - All Occupations,USA,61600.0
3,Ann 50th %ile Wage1,Total - All Occupations,USA,45700.0
4,LQ,Total - All Occupations,USA,1.0


In [112]:
data = data.pivot(index = ['Occupation', 'NAME'], columns = 'Value')['IDK']

In [113]:
data = data.reset_index(drop = False)

In [114]:
data.head()

Value,Occupation,NAME,3 Year Forecast Growth Rate,3 Year Growth Demand,Ann 50th %ile Wage1,Annual Mean Wage1,Empl (Place of Residence),Empl (Place of Work),LQ,Percent of Total,Total 3 Year Forecast Demand
0,Business & Financial,"Adams city, TN",0.044,0.0,84800.0,94300.0,52.0,8.0,0.86,11.940299,2.0
1,Business & Financial,"Ashland City town, TN",0.031,17.0,79700.0,88900.0,165.0,537.0,0.81,10.731415,161.0
2,Business & Financial,"Belle Meade city, TN",0.053,7.0,94400.0,109900.0,338.0,134.0,0.76,10.159212,46.0
3,Business & Financial,"Berry Hill city, TN",0.024,61.0,87800.0,101800.0,136.0,2585.0,1.07,14.121053,741.0
4,Business & Financial,"Bethpage CDP, TN",0.035,0.0,87800.0,100300.0,11.0,4.0,0.45,6.060606,1.0


In [115]:
data = data.set_index('Occupation').transpose()

In [116]:
data.head()

Occupation                  Business & Financial   Business & Financial  Business & Financial Business & Financial Business & Financial  \
Value                                                                                                                                     
NAME                              Adams city, TN  Ashland City town, TN  Belle Meade city, TN  Berry Hill city, TN     Bethpage CDP, TN   
3 Year Forecast Growth Rate                0.044                  0.031                 0.053                0.024                0.035   
3 Year Growth Demand                         0.0                   17.0                   7.0                 61.0                  0.0   
Ann 50th %ile Wage1                      84800.0                79700.0               94400.0              87800.0              87800.0   
Annual Mean Wage1                        94300.0                88900.0              109900.0             101800.0             100300.0   

Occupation                  Business & Financial Business & Financial Business & Financial       Business & Financial Business & Financial  \
Value                                                                                                                                        
NAME                           Bransford CDP, TN   Brentwood city, TN       Burns town, TN  Castalian Springs CDP, TN  Cedar Hill city, TN   
3 Year Forecast Growth Rate                 0.03                0.087                0.033                       0.03                0.043   
3 Year Growth Demand                         0.0               1080.0                  1.0                        0.0                  1.0   
Ann 50th %ile Wage1                      81700.0              89400.0              79000.0                    83000.0              70100.0   
Annual Mean Wage1                        92900.0             103200.0              87000.0                    94400.0              77700.0   

Occupation                  Business & Financial        Business & Financial Business & Financial  Business & Financial Business & Financial  \
Value                                                                                                                                          
NAME                          Charlotte town, TN  Cheatham County, Tennessee   Christiana CDP, TN  Clarksville city, TN    Columbia city, TN   
3 Year Forecast Growth Rate                0.029                       0.029                0.062                 0.057                0.069   
3 Year Growth Demand                         4.0                        37.0                  2.0                 301.0                181.0   
Ann 50th %ile Wage1                      75000.0                     81800.0              86800.0               74400.0              77900.0   
Annual Mean Wage1                        82700.0                     91200.0              99600.0               83700.0              86600.0   

Occupation                  Business & Financial Business & Financial   Business & Financial      Business & Financial        Business & Financial  \
Value                                                                                                                                                
NAME                         Coopertown town, TN   Cottontown CDP, TN  Cross Plains city, TN  Cumberland City town, TN  Davidson County, Tennessee   
3 Year Forecast Growth Rate                0.041                0.055                  0.029                     0.017                       0.042   
3 Year Growth Demand                         6.0                  0.0                    1.0                       2.0                      3274.0   
Ann 50th %ile Wage1                      81800.0              78400.0                86000.0                   77000.0                     89800.0   
Annual Mean Wage1                        90900.0              88900.0                95700.0                   81900.0    

In [117]:
#rename industries
data = data.rename(columns = {'Healthcare Practitioners and Technical Occupations (29-0000)': 'Healthcare Practitioners & Technical',
                              'Healthcare Support Occupations (31-0000)': 'Healthcare Support', 
                              'Protective Service Occupations (33-0000)': 'Protective Services',
                              'Production Occupations (51-0000)': 'Production', 
                              'Transportation and Material Moving Occupations (53-0000)': 'Transportation & Material Moving'}).transpose()

In [118]:
data = data.reset_index(drop = False)

In [119]:
data.head()

Value,Occupation,NAME,3 Year Forecast Growth Rate,3 Year Growth Demand,Ann 50th %ile Wage1,Annual Mean Wage1,Empl (Place of Residence),Empl (Place of Work),LQ,Percent of Total,Total 3 Year Forecast Demand
0,Business & Financial,"Adams city, TN",0.044,0.0,84800.0,94300.0,52.0,8.0,0.86,11.940299,2.0
1,Business & Financial,"Ashland City town, TN",0.031,17.0,79700.0,88900.0,165.0,537.0,0.81,10.731415,161.0
2,Business & Financial,"Belle Meade city, TN",0.053,7.0,94400.0,109900.0,338.0,134.0,0.76,10.159212,46.0
3,Business & Financial,"Berry Hill city, TN",0.024,61.0,87800.0,101800.0,136.0,2585.0,1.07,14.121053,741.0
4,Business & Financial,"Bethpage CDP, TN",0.035,0.0,87800.0,100300.0,11.0,4.0,0.45,6.060606,1.0


In [120]:
data = data.set_index('NAME').transpose()

In [121]:
data = data.rename(columns = {'Adams city, TN': 'Adams', 'Ashland City town, TN': 'Ashland City', 'Belle Meade city, TN': 'Belle Meade',
                                    'Berry Hill city, TN': 'Berry Hill', 'Brentwood city, TN': 'Brentwood', 'Burns town, TN': 'Burns',
                                    'Cedar Hill city, TN': 'Cedar Hill', 'Charlotte town, TN': 'Charlotte',
                                    'Cheatham County, Tennessee': 'Cheatham County', 'Clarksville city, TN': 'Clarksville',
                                    'Columbia city, TN': 'Columbia', 'Coopertown town, TN': 'Coopertown',
                                    'Cross Plains city, TN': 'Cross Plains', 'Cumberland City town, TN': 'Cumberland City',
                                    'Davidson County, Tennessee': 'Davidson County', 'Dickson city, TN': 'Dickson',
                                    'Dickson County, Tennessee': 'Dickson County', 'Dover city, TN': 'Dover',
                                    'Eagleville city, TN': 'Eagleville', 'Erin city, TN': 'Erin', 'Fairview city, TN': 'Fairview',
                                    'Forest Hills city, TN': 'Forest Hills', 'Franklin city, TN': 'Franklin', 'Gallatin city, TN': 'Gallatin',
                                    'Goodlettsville city, TN': 'Goodlettsville', 'Greenbrier town, TN': 'Greenbrier',
                                    'Hartsville/Trousdale County, TN': 'Hartsville/Trousdale', 'Hendersonville city, TN': 'Hendersonville',
                                    'Houston County, Tennessee': 'Houston County', 
                                    'Kingston Springs town, TN': 'Kingston Springs', 'La Vergne city, TN': 'La Vergne',
                                    'Lebanon city, TN': 'Lebanon', 'McEwen city, TN': 'McEwen',
                                    'Millersville city, TN': 'Millersville', 'Mitchellville city, TN': 'Mitchellville',
                                    'Montgomery County, TN': 'Montgomery County', 'Mount Juliet city, TN': 'Mount Juliet',
                                    'Mount Pleasant city, TN': 'Mount Pleasant', 'Murfreesboro city, TN': 'Murfreesboro',
                                    'Nashville-Davidson metropolitan government (balance), TN': 'Nashville', 
                                    'New Johnsonville city, TN': 'New Johnsonville', 'Nolensville town, TN': 'Nolensville',
                                    'Oak Hill city, TN': 'Oak Hill', 'Pegram town, TN': 'Pegram', 'Pleasant View city, TN': 'Pleasant View',
                                    'Portland city, TN': 'Portland', 'Ridgetop city, TN': 'Ridgetop',
                                    'Rutherford County, TN': 'Rutherford County', 'Slayden town, TN': 'Slayden', 'Smyrna town, TN': 'Smyrna',
                                    'Spring Hill city, TN': 'Spring Hill', 'Springfield city, TN': 'Springfield',
                                    'Stewart County, Tennessee': 'Stewart County', 'Sumner County, Tennessee': 'Sumner County',
                                    'Tennessee Ridge town, TN': 'Tennessee Ridge', "Thompson's Station town, TN": "Thompson's Station",
                                    'Trousdale County, Tennessee': 'Trousdale County', 'Vanleer town, TN': 'Vanleer', 'Watertown city, TN': 'Watertown',
                                    'Waverly city, TN': 'Waverly', 'Westmoreland town, TN': 'Westmoreland', 'White Bluff town, TN': 'White Bluff',
                                    'White House city, TN': 'White House', 'Williamson County, Tennessee': 'Williamson County',
                                    'Wilson County, Tennessee': 'Wilson County'})
data = data.transpose()
data = data.reset_index()

In [125]:
data.head()

Value,NAME,Occupation,3 Year Forecast Growth Rate,Median Annual Wages,Employment,Location Quotient,Percent of Total
0,Adams,Business & Financial,4.4,84800.0,8.0,0.86,11.940299
1,Ashland City,Business & Financial,3.1,79700.0,537.0,0.81,10.731415
2,Belle Meade,Business & Financial,5.3,94400.0,134.0,0.76,10.159212
3,Berry Hill,Business & Financial,2.4,87800.0,2585.0,1.07,14.121053
4,"Bethpage CDP, TN",Business & Financial,3.5,87800.0,4.0,0.45,6.060606


In [123]:
data = data.rename(columns = {'Total 3 Year Forecast Demand': '3 Year Forecast Demand', '3 Year Growth Demand': '3 Year Forecast Growth Demand', 
                              'Empl (Place of Work)': 'Employment', 'LQ': 'Location Quotient', 'Annual Mean Wage1': 'Average Annual Wages', 
                              'Ann 50th %ile Wage1': 'Median Annual Wages'
                             })

In [124]:
data['3 Year Forecast Growth Rate'] = data['3 Year Forecast Growth Rate'] * 100
data = data.drop(columns = ['3 Year Forecast Demand', '3 Year Forecast Growth Demand', 'Empl (Place of Residence)', 'Average Annual Wages'])

In [126]:
data.to_csv('../../Outputs/JOBSEQ_OCCUPATIONALEMPSTATSCURRENT_ALLGEOS.csv', index = False)